In [ ]:
!pip install transformers
!pip install -U sentence-transformers
!pip install gensim

     |████████████████████████████████| 1.3MB 21.2MB/s 
     |████████████████████████████████| 890kB 48.8MB/s 
     |████████████████████████████████| 1.1MB 47.8MB/s 
     |████████████████████████████████| 2.9MB 46.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=f2bee8a93e4599834bccbff1268addd067390dbba2ebc818d54a21838da415a1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 71kB 7.9MB/s 
     |████████████████████████████████| 1.1MB 21.6MB/s 
     |████████████████████████████████| 3.0MB 43.3MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.8-cp36-none-any.whl size=101996 sha256=11aa57d98c22bc64b41967b15567b472f921f58b218389b7664d5a582ad1b7d9
  Stored in directory: /root/.cache/pip/wheels/27/ec/b3/d12cc8e4daf77846db6543033d3a5642f204c0320b15945647
Successfully built sent

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/'My Drive'/'CS229'
# %rm -rf fid_test_pix2pix_3
%ls

/content/drive/.shortcut-targets-by-id/1sPYcNDVL_owY8VCbxPtp6HF2oVELOYJo/CS229
 atis/                                     generate_bert_encodings.py
 atis_logs/                                generate_word2vec_encodings.py
 atis_logs_freeze/                         load_data_1.py
 atis_results/                             load_data.py
'Copy of Fine-tune BERT (SNIPS).ipynb'     LSTM_Model/
'CS 229 Project Ideas.gdoc'               'Project Planning.gdoc'
 data/                                     __pycache__/
 data.gsheet                               Resources.gdoc
 embeddings/                               snips/
'Fine-tune BERT (ATIS) (Freeze).ipynb'     snips_distil_logs/
'Fine-tune BERT (ATIS).ipynb'              snips_electra_logs/
'Fine-tune BERT.ipynb'                     snips_logs/
'Fine-tune BERT (SNIPS) - Copy.ipynb'      snips_logs_copy/
'Fine-tune BERT (SNIPS) (Freeeze).ipynb'   snips_logs_freeeze/
'Fine-tune BERT (SNIPS).ipynb'             snips_longformer_logs/
'Fine-tun

In [ ]:
import os, sys
import argparse
import numpy as np
import torch
import transformers
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, f1_score
from load_data import *

In [ ]:
num_labels = 7
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels, return_dict=True)
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
DATASET = 'snips'
TRAIN_PATH = f'./{DATASET}/{DATASET}_train_actual.csv'
TEST_PATH = f'./{DATASET}/{DATASET}_test_actual.csv'

In [ ]:
data_loader = DataLoader(TRAIN_PATH, None, TEST_PATH)
data_loader.split_train_valid(valid_size=0.05, keep_class_ratios=True)
X_train, y_train = data_loader.get_train_data()
X_valid, y_valid = data_loader.get_valid_data()
X_test, y_test = data_loader.get_test_data()

In [ ]:
X_train = tokenizer(X_train, truncation=True, padding=True)
X_valid = tokenizer(X_valid, truncation=True, padding=True)
X_test = tokenizer(X_test, truncation=True, padding=True)

In [ ]:
class IntentDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        # sample_X = self.X[idx]
        # sample_y = self.y[idx]
        # sample = {
        #     'text': sample_X,
        #     'label': sample_y,
        # }
        # return sample
        item = {key: torch.tensor(val[idx]) for key, val in self.X.items()}
        item['labels'] = torch.tensor(self.y[idx])
        return item

In [ ]:
# def tokenize(batch):
#     return tokenizer(batch['text'], padding=True, truncation=True)

# train_dataset = IntentDataset(X_train, y_train)
# train_dataset = train_dataset.map(tokenize, batched=True, batch_size=batch_size)
# # test_dataset = test_dataset.map(tokenize, batched=True, batch_size=batch_size)
# train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
# # test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
train_dataset = IntentDataset(X_train, y_train)
valid_dataset = IntentDataset(X_valid, y_valid)
test_dataset = IntentDataset(X_test, y_test)

In [ ]:
# def compute_metrics(pred):
#     labels = pred.label_ids
#     preds = pred.predictions.argmax(-1)
#     precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
#     acc = accuracy_score(labels, preds)
#     return {
#         'accuracy': acc,
#         'f1': f1,
#         'precision': precision,
#         'recall': recall
#     }

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
    }

In [ ]:
batch_size = 16
lr = 1e-5
num_epochs = 20
weight_decay = 0.0001

In [ ]:
model.train()

# for param in model.base_model.parameters():
#     param.requires_grad = False

training_args = TrainingArguments(
    output_dir=f'./{DATASET}_results',
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=1,
    learning_rate=lr,
    warmup_steps=500,
    weight_decay=weight_decay,
    evaluate_during_training=True,
    logging_dir=f'./{DATASET}_logs/{num_epochs}_{batch_size}_{lr}_{weight_decay}',
    load_best_model_at_end=True,
    metric_for_best_model='eval_accuracy',
    save_steps=1,
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset
)

/usr/local/lib/python3.6/dist-packages/transformers/training_args.py:332: FutureWarning: The `evaluate_during_training` argument is deprecated in favor of `evaluation_strategy` (which has more options)
  FutureWarning,


In [ ]:
trainer.train()

{'loss': 1.111192138671875, 'learning_rate': 1e-05, 'epoch': 0.6105006105006106, 'total_flos': 210216236160000, 'step': 500}



{'eval_loss': 0.12533477012985858, 'eval_accuracy': 0.9753623188405797, 'eval_f1': 0.9753333005721146, 'epoch': 0.6105006105006106, 'total_flos': 210216236160000, 'step': 500}



{'loss': 0.0710628662109375, 'learning_rate': 9.685138539042822e-06, 'epoch': 1.221001221001221, 'total_flos': 420169702024800, 'step': 1000}



{'eval_loss': 0.061694530327804385, 'eval_accuracy': 0.9869565217391304, 'eval_f1': 0.9868874512403748, 'epoch': 1.221001221001221, 'total_flos': 420169702024800, 'step': 1000}
{'loss': 0.0528070068359375, 'learning_rate': 9.370277078085643e-06, 'epoch': 1.8315018315018317, 'total_flos': 630385938184800, 'step': 1500}



{'eval_loss': 0.050671143674939544, 'eval_accuracy': 0.9869565217391304, 'eval_f1': 0.9869313333173708, 'epoch': 1.8315018315018317, 'total_flos': 630385938184800, 'step': 1500}



...


{'eval_loss': 0.11483302145098948, 'eval_accuracy': 0.9898550724637681, 'eval_f1': 0.9898247063975792, 'epoch': 18.315018315018314, 'total_flos': 6301757219486400, 'step': 15000}
{'loss': 0.0, 'learning_rate': 5.541561712846348e-07, 'epoch': 18.925518925518926, 'total_flos': 6511973455646400, 'step': 15500}



{'eval_loss': 0.11507141180706534, 'eval_accuracy': 0.9898550724637681, 'eval_f1': 0.9898247063975792, 'epoch': 18.925518925518926, 'total_flos': 6511973455646400, 'step': 15500}



{'loss': 1.220703125e-07, 'learning_rate': 2.3929471032745596e-07, 'epoch': 19.536019536019538, 'total_flos': 6721926921511200, 'step': 16000}



{'eval_loss': 0.11532607485196751, 'eval_accuracy': 0.9898550724637681, 'eval_f1': 0.9898247063975792, 'epoch': 19.536019536019538, 'total_flos': 6721926921511200, 'step': 16000}




TrainOutput(global_step=16380, training_loss=0.041240672576121795)

In [ ]:
trainer.evaluate(test_dataset)


{'eval_loss': 0.05444581236457452, 'eval_accuracy': 0.99, 'eval_f1': 0.9900249238577276, 'epoch': 20.0, 'total_flos': 6881428490697600, 'step': 16380}


{'epoch': 20.0,
 'eval_accuracy': 0.99,
 'eval_f1': 0.9900249238577276,
 'eval_loss': 0.05444581236457452,
 'total_flos': 6881428490697600}